In [ ]:
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from lane_lines.calibration import load_calibration
from lane_lines.Image_Processor import Image_Processor
from lane_lines.Line import Line

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

cam_matrix, dist_coeff = load_calibration('camera_cal')

src = np.float32([[238, 688] , [1071, 688] ,[595, 449],[687, 449]])
dst = np.float32([[280, 720] , [1000, 720] ,[280, 0],[1000, 0]])
M = cv2.getPerspectiveTransform(src, dst)
M_inv = cv2.getPerspectiveTransform(dst, src)
image_processor = Image_Processor(cam_matrix, dist_coeff, M, M_inv)


In [ ]:
video_output = 'output_videos/project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(image_processor.process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(video_output, audio=False)

## Undistort Image

In [ ]:
image_processor.new_image(mpimg.imread('test_images/test1.jpg'))
image_processor.undistort()

img_undist = image_processor.img_undist#cv2.undistort(img_orig, cam_matrix, dist_coeff, None, cam_matrix)    
                   
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(image_processor.img_orig)#,cmap='gray')
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(img_undist, cmap='gray')
ax2.set_title('Undistorted Image', fontsize=50)
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/test1_undist.jpg')

## Thresholding

In [ ]:
image_processor.calc_abs_gradient_binary(kernel_sizes=(3,7,11), thresh=(60, 150))
image_processor.calc_dir_gradient_binary(kernel_sizes=(3,7,11), thresh=(30, 60), degrees=True)
image_processor.calc_hls_thresh_binary(channel='s', thresh=(170, 255))
image_processor.calc_combined_binary()
# Plot the result
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(24, 18))
f.tight_layout()
ax1.imshow(image_processor.img_undist)
ax1.set_title('Input Image', fontsize=50)
ax2.imshow(image_processor.abs_gradient_binary * image_processor.dir_gradient_binary, cmap='gray')
ax2.set_title('Thresholded Gradient', fontsize=50)
ax3.imshow(image_processor.hls_thresh_binary, cmap='gray')
ax3.set_title('Thresholded s Channel', fontsize=50)
ax4.imshow(image_processor.color_binary())#, cmap='gray')
ax4.set_title('Stacked Threshold', fontsize=50)
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/test1_binary_combo_example.jpg')

## Perspective Transform

In [ ]:
src = np.float32([[238, 688] , [1071, 688] ,[595, 449],[687, 449]])
dst = np.float32([[280, 720] , [1000, 720] ,[280, 0],[1000, 0]])
M = cv2.getPerspectiveTransform(src, dst)
M_inv = cv2.getPerspectiveTransform(dst, src)
image_processor2 = Image_Processor(cam_matrix, dist_coeff, M, M_inv)
image_processor2.new_image(mpimg.imread('test_images/straight_lines1.jpg'))
image_processor2.undistort()
warped = image_processor2.get_warped_perspective_img_undist()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(image_processor2.img_undist)#, cmap='gray')
ax1.set_title('Original  Image', fontsize=50)
ax1.plot([src[0][0], src[1][0]], [src[0][1], src[1][1]], color='red')
ax1.plot([src[1][0], src[3][0]], [src[1][1], src[3][1]], color='red')
ax1.plot([src[3][0], src[2][0]], [src[3][1], src[2][1]], color='red')
ax1.plot([src[0][0], src[2][0]], [src[0][1], src[2][1]], color='red')
ax2.imshow(warped)#, cmap='gray')
ax2.set_title('Warped Image', fontsize=50)
ax2.axvline(280, color='red')
ax2.axvline(310, color='red')
ax2.axvline(970, color='red')
ax2.axvline(1000, color='red')
plt.savefig('output_images/straight_lines1_warped_example.jpg')

In [ ]:
image_processor.warp_perspective_binary()
img_sliding_window = image_processor.find_lane_pixels_sliding_window(return_image=True)
y_values = np.linspace(0,720, 721)
x1_sl = image_processor.line_left.get_poly_pixel_x_values(y_values)
x2_sl = image_processor.line_right.get_poly_pixel_x_values(y_values)
img_poly = image_processor.find_lane_pixels_poly(return_image=True)
x1_poly = image_processor.line_left.get_poly_pixel_x_values(y_values)
x2_poly = image_processor.line_right.get_poly_pixel_x_values(y_values)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img_sliding_window, cmap='gray')
ax1.plot(x1_sl, y_values)
ax1.plot(x2_sl, y_values)
ax1.set_title('Sliding Window', fontsize=50)
ax2.imshow(img_poly, cmap='gray')
ax2.plot(x1_poly, y_values)
ax2.plot(x2_poly, y_values)
ax2.set_title('Look Ahead', fontsize=50)
plt.savefig('output_images/lane_line_identification.jpg')

print(image_processor.line_left.measure_curvature_real())
print(image_processor.line_right.measure_curvature_real())

In [ ]:
image_processor.find_lane_line_pixels()
plt.figure(num=None, figsize=(12, 9))
plt.imshow(image_processor.draw_lane())
plt.savefig('output_images/example_output.jpg')